# Loading the libraries

In [291]:
import csv
import pandas as pd
import os
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
import random
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import vgg16
from tensorflow.keras import Model

# Creating Expression Image Dictionary

In [2]:
data={}
with open("data\\facial_expressions\\data\\legend.csv") as f:
    reader=csv.reader(f)
    next(reader)
    for row in reader:
        key=row[2].lower()  
        if key in data:
            data[key].append(row[1])
        else:
            data[key]=[row[1]]
    


# Creating and Populating Directories for ImageDataGenerator Class

### Total number of categories

In [3]:
data.keys()

dict_keys(['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'contempt'])

### Making the directories

In [4]:
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [5]:
for emotion in data.keys():
    os.mkdir(os.path.join('master_data/training/',emotion))
    os.mkdir(os.path.join('master_data/testing/', emotion))
    

# Copying the files

In [6]:
split_size=0.2
faulty_images_fix=0.1  


for emotion, images in data.items():
    temp=int(faulty_images_fix*len(images))
    images=images[temp:]
    
    
    random.shuffle(images)
    test_size= int(split_size*len(images))
    
    test_images=images[:test_size]  # First 20 % images are for testing
    train_images=images[test_size:]  #  Last 80% images are for training
    
    
    for image in train_images:
        source= os.path.join('data\\facial_expressions\\images',image)
        #print(source)
        destination=os.path.join('master_data\\training',emotion,image)
        #print(destination)
        copyfile(source,destination)
    
    for image in test_images:
        source= os.path.join('data\\facial_expressions\\images',image)
        #print(source)
        destination=os.path.join('master_data\\testing',emotion,image)
        #print(destination)
        copyfile(source,destination)        


# Creating the model

In [7]:
model=tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPool2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
    
])

In [8]:
model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

# Using Image DataGenerator Class

In [10]:
# data source
train_dir="master_data\\training"
test_dir="master_data\\testing"

In [11]:
train_data_gen=ImageDataGenerator(rescale=1.0/255)

In [12]:
train_data_generater=train_data_gen.flow_from_directory(
                                                    train_dir, 
                                                    target_size=(100,100),
                                                    class_mode='categorical',              
                                                    batch_size=128                    )

Found 9861 images belonging to 8 classes.


In [13]:
train_data_generater.num_classes

8

In [14]:
test_data_gen=ImageDataGenerator(rescale=1.0/255)

In [15]:
test_data_generator=test_data_gen.flow_from_directory(test_dir, target_size=(100,100), class_mode='categorical', batch_size=128)

Found 2461 images belonging to 8 classes.


# Adding the early stopping

In [16]:
es=EarlyStopping(monitor="val_accuracy", patience=2, min_delta=0.01)

# Training and validation

In [17]:
#model.fit_generator(train_data_generater, epochs=10, verbose=True, callbacks=[es])
model.fit(train_data_generater, epochs=15, verbose=True, callbacks=[es], validation_data=test_data_generator)

Epoch 1/15
78/78 [==============================] - 88s 1s/step - loss: 1.5016 - accuracy: 0.4800 - val_loss: 1.0264 - val_accuracy: 0.5022
Epoch 2/15
78/78 [==============================] - 30s 384ms/step - loss: 1.0265 - accuracy: 0.5397 - val_loss: 0.9488 - val_accuracy: 0.5729
Epoch 3/15
78/78 [==============================] - 30s 385ms/step - loss: 0.8615 - accuracy: 0.6889 - val_loss: 0.7747 - val_accuracy: 0.7298
Epoch 4/15
78/78 [==============================] - 30s 383ms/step - loss: 0.7832 - accuracy: 0.7165 - val_loss: 0.7280 - val_accuracy: 0.7452
Epoch 5/15
78/78 [==============================] - 30s 383ms/step - loss: 0.7558 - accuracy: 0.7338 - val_loss: 0.7428 - val_accuracy: 0.7473
Epoch 6/15
78/78 [==============================] - 30s 383ms/step - loss: 0.7136 - accuracy: 0.7503 - val_loss: 0.6922 - val_accuracy: 0.7611
Epoch 7/15
78/78 [==============================] - 30s 385ms/step - loss: 0.6983 - accuracy: 0.7513 - val_loss: 0.6806 - val_accuracy: 0.7720
Ep

### We are getting 77% validation accuracy

# Using Transfer Learning

In [ ]:
keras.applications.vgg16.preprocess_input

In [281]:
Input_Shape=(224,224)

In [284]:
vgg=VGG16(include_top=False, input_shape=Input_Shape+(3,))

58900480/58889256 [==============================] - 6s 0us/step


In [286]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [287]:
for layer in vgg.layers:
    layer.trainable=False

#### Adding the last layers

In [288]:
x=Flatten()(vgg.output)

In [289]:
data.keys()

dict_keys(['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'contempt'])

In [290]:
prediction=Dense(8,activation='softmax')(x)

#### Creating the model object

In [292]:
model=Model(inputs=vgg.input,outputs=prediction)

In [293]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Compiling the model

In [295]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

## Image Data generators for Transfer learning

In [298]:
train_data_gen=ImageDataGenerator(rescale=1.0/255)
train_data_generater=train_data_gen.flow_from_directory(
                                                    train_dir, 
                                                    target_size=(Input_Shape),
                                                    class_mode='categorical',              
                                                    batch_size=128                    )

test_data_gen=ImageDataGenerator(rescale=1.0/255)
test_data_generator=test_data_gen.flow_from_directory(test_dir, target_size=(Input_Shape), class_mode='categorical', batch_size=128)


Found 9861 images belonging to 8 classes.
Found 2461 images belonging to 8 classes.


## Training the model

In [301]:
model.fit(train_data_generater,validation_data=test_data_generator,callbacks=[es], verbose=True, epochs=5)

Epoch 1/5
78/78 [==============================] - 793s 10s/step - loss: 0.9431 - accuracy: 0.8256 - val_loss: 0.8791 - val_accuracy: 0.7834
Epoch 2/5
78/78 [==============================] - 784s 10s/step - loss: 0.5151 - accuracy: 0.8579 - val_loss: 0.7583 - val_accuracy: 0.8094
Epoch 3/5
78/78 [==============================] - 785s 10s/step - loss: 0.9210 - accuracy: 0.8706 - val_loss: 1.1056 - val_accuracy: 0.7765
Epoch 4/5
78/78 [==============================] - 784s 10s/step - loss: 0.5722 - accuracy: 0.8542 - val_loss: 1.0703 - val_accuracy: 0.7879


# Saving the model

In [303]:
model.save('Trandsfer Learning Facial Recognition.h5')